In [5]:
# load the refCOCOg dataset


!pip install gdown
import gdown

url = "https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq"
gdown.download(url)

!tar -xvf /content/refcocog.tar.gz

!rm /content/refcocog.tar.gz

import pickle
import numpy as np
import json
import matplotlib.pyplot as plt
import os

from PIL import Image

with open("./refcocog/annotations/refs(umd).p", "rb") as fp:
  annotations = pickle.load(fp)

  train_data = sorted([d for d in annotations if d["split"]=="train"], key=lambda d: d["file_name"])
  val_data = sorted([d for d in annotations if d["split"]=="val"], key=lambda d: d["image_id"])
  test_data = sorted([d for d in annotations if d["split"]=="test"], key=lambda d: d["image_id"])

  entry = train_data[0]

  print(json.dumps(entry, indent=4, default=str))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
To: /content/refcocog.tar.gz
100%|██████████| 13.5G/13.5G [05:14<00:00, 42.9MB/s]


Output streaming troncato alle ultime 5000 righe.
./refcocog/images/COCO_train2014_000000419396.jpg
./refcocog/images/COCO_train2014_000000433336.jpg
./refcocog/images/COCO_train2014_000000567523.jpg
./refcocog/images/COCO_train2014_000000451554.jpg
./refcocog/images/COCO_train2014_000000439855.jpg
./refcocog/images/COCO_train2014_000000044594.jpg
./refcocog/images/COCO_train2014_000000194276.jpg
./refcocog/images/COCO_train2014_000000272463.jpg
./refcocog/images/COCO_train2014_000000485426.jpg
./refcocog/images/COCO_train2014_000000002774.jpg
./refcocog/images/COCO_train2014_000000346160.jpg
./refcocog/images/COCO_train2014_000000199829.jpg
./refcocog/images/COCO_train2014_000000210057.jpg
./refcocog/images/COCO_train2014_000000544117.jpg
./refcocog/images/COCO_train2014_000000440673.jpg
./refcocog/images/COCO_train2014_000000006151.jpg
./refcocog/images/COCO_train2014_000000514391.jpg
./refcocog/images/COCO_train2014_000000247459.jpg
./refcocog/images/COCO_train2014_000000141257.jpg


In [6]:
!cp ./refcocog /content/drive/My Drive/

cp: target 'Drive/' is not a directory


In [21]:
import os
import json

from torch.utils.data import Dataset

class RefCOCOg(Dataset):
    def __init__(self, refs, annotations, split="train"):

        self.dataset = [{"file_name": os.path.join("./refcocog/images/", f'{"_".join(elem["file_name"].split("_")[:3])}.jpg'),
                            "caption": [i["raw"] for i in elem["sentences"]],
                            "ann_id": int(elem["file_name"].split("_")[3][:-4]),
                            "bbox": annotations[int(elem["file_name"].split("_")[3][:-4])]}
                        for elem in [d for d in refs if d["split"]==split]]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]
    
    def __call__(self, idx):
        print(json.dumps(self.dataset[idx], indent=4))


# Load refs and annotations
import pickle

with open("./refcocog/annotations/refs(umd).p", "rb") as fp:
  refs = pickle.load(fp)

with open("./refcocog/annotations/instances.json", "rb") as fp:
  data = json.load(fp)
  annotations = dict(sorted({ann["id"]: ann["bbox"] for ann in data["annotations"]}.items()))

# load the train dataset
# test_dataset_full = RefCOCOg(refs=refs, annotations=annotations, split="test")
# test_dataset_full = RefCOCOg(refs=refs, annotations=annotations, split="val")
test_dataset_full = RefCOCOg(refs=refs, annotations=annotations, split="train")


print('len test datasets:',sum([len(i['caption']) for i in test_dataset_full]))

len test datasets: 80512


In [8]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
base_path_save = '/content/drive/MyDrive/'

In [12]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-f90kj43u
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-f90kj43u
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369370 sha256=70a162468942db61bbe885ebf5a1653ad5b7dd165a77d90cd5f7ae94a3b2a29e
  Stored in directory: /tmp/pip-ephem-wheel-cache-65in_3om/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [14]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.4/591.4 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 25.1 MB/s eta 0:00:00


In [22]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms as T
import clip
from matplotlib.patches import Rectangle
import numpy as np
from PIL import Image
import time
import pandas as pd
from ultralytics import YOLO


# Number of images to test
N_img = 10

# Model
del model
# model = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True) # yolo5l6
model = YOLO("yolov8x.pt")

print(clip.available_models())

clip_model, preprocess = clip.load("ViT-B/32", device="cuda")

def get_crops(yolov5_df, image):
    """ Get crops from yolov8_df 

    Args:
        yolov5_df (pd.DataFrame): DataFrame with yolov5 predictions
        image (np.array): image as np.array

    Returns:
        list: crops
    
    """
    crops = []
    image = image

    for box in yolov5_df.values:
        x_min, y_min, x_max, y_max, confidence = box[:5]

        if confidence > 0.5:
            crop = image[int(y_min):int(y_max), int(x_min):int(x_max)]
            crop = Image.fromarray(crop)
            crops.append(crop)
        
    # add the entire image
    crops.append(Image.fromarray(image))

    return crops

def get_crops_preprocessed_for_clip(yolov8_df, image):
    """ Get crops from yolov5_df and preprocess them for CLIP

    Args:
        yolov8_df (pd.DataFrame): DataFrame with yolov5 predictions
        image (np.array): image as np.array

    Returns:
        torch.tensor: preprocessed crops
    
    """
    crops = []
    image = image

    for box in yolov8_df.values:
        x_min, y_min, x_max, y_max, confidence = box[:5]

        if confidence > 0.5:
            crop = image[int(y_min):int(y_max), int(x_min):int(x_max)]
            crop = Image.fromarray(crop)
            crop = preprocess(crop).cuda()
            crops.append(crop)
        
    # add the entire image
    crops.append(preprocess(Image.fromarray(image)).cuda())

    return torch.stack(crops)

def plot_image_yolov8(results, image):
    for i in results.xyxy[0]:
        if i[4] > 0.5: # if confidence is greater than 0.5
            # Create figure and axes
            _, ax = plt.subplots()

            boxes = i[:4]

            # Display the image
            ax.imshow(image)

            # Create a Rectangle patch
            x_min, y_min, width, height = boxes.tolist()
            ax.add_patch(Rectangle((x_min, y_min), width-x_min, height-y_min, linewidth=1, edgecolor='r', facecolor='none'))

            ax.axes.xaxis.set_visible(False)
            ax.axes.yaxis.set_visible(False)

            plt.show()

def get_dict_clip_emb_texscores(clip_model, img_preproc, text, idxs):
    """ Get dictionary with CLIP embeddings and text scores

    Args:
        clip_model (CLIP): CLIP model
        img_preproc (torch.tensor): preprocessed crops
        text (list): tokenized text
    
    Returns:
        dict: dictionary with CLIP embeddings and text scores
    
    """
    d_emb_texscores = {}

    for idx, images, captions in zip(idxs, img_preproc, text):
        temp = {}
        with torch.no_grad():
            # get CLIP embeddings
            image_features = clip_model.encode_image(images)
            text_features = clip_model.encode_text(captions)

        temp['image_emb'] = image_features.cpu()
        temp['text_emb'] = text_features.cpu()

        # normalize
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        temp['text_similarity'] = (100.0 * text_features @ image_features.T).softmax(dim=-1).cpu()

        # print("Label probs:", similarity.cpu().numpy())

        d_emb_texscores[idx] = temp

    return d_emb_texscores


# Inference
def load_image_plt(ref):
    return plt.imread('./refcocog/images/'+ref['file_name'].split('/')[-1])   
def load_image_pil(ref):
    return Image.open('./refcocog/images/'+ref['file_name'].split('/')[-1]) 

#####################################################

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#####################################################

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

#####################################################

def rescale_bboxes(out_bbox, size, device):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32).to(device)
    return b

#####################################################

def detect(im, model, transform, device):
    # mean-std normalize the input image (batch-size: 1)
    img = transform(im).unsqueeze(0).to(device)

    # demo model only support by default images with aspect ratio between 0.5 and 2
    # if you want to use images with an aspect ratio outside this range
    # rescale your image so that the maximum size is at most 1333 for best results
    # assert img.shape[-2] <= 1600 and img.shape[-1] <= 1600, 'demo model only supports images up to 1600 pixels on each side'

    # propagate through the model
    outputs = model(img)

    # keep only predictions with 0.7+ confidence
    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.5
    # keep = probas.max(-1).values > 0.7

    # convert boxes from [0; 1] to image scales
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size, device)
    return probas[keep], bboxes_scaled

#####################################################

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [ ]:
import pickle

# initialize steps for the loop
min_image = 28000
max_image = len(test_dataset_full)
# batch of 50 images
N_images_batches = 50
steps = int(max_image/N_images_batches)
steps = np.linspace(min_image,max_image,steps).astype(int)

dictionary_full = {}

for m, M in zip(steps[:max_image-1], steps[1:]):
    
    print('step:',m,M)
    # initialize patch
    test_dataset = test_dataset_full[m:M]
    N_img = len(test_dataset)

    start = time.time()
    results = model([load_image_pil(test_dataset[i]) for i in range(N_img)])

    # Results
    # results.print()

    # store the df
    # yolov5
    # results = [i for i in results.pandas().xyxy]
    # yolov8
    results = [pd.DataFrame(result.boxes.boxes.tolist(), columns=['x_min', 'y_min', 'x_max', 'y_max', 'confidence', 'class']) for result in results]

    # plot_image_yolov8(results, np.array(load_image_plt(test_dataset[0])))
        
    img_preproc = [
        get_crops_preprocessed_for_clip(yolov8_df, image) 
            for yolov8_df, image in zip(results, [load_image_plt(test_dataset[i]) for i in range(N_img)])
            ]

    # img_preproc[0].shape

    # tokenize the text with clip
    text = [clip.tokenize([caption for caption in test_dataset[idx]['caption']]).cuda() for idx in range(N_img)]
    captions = [[caption for caption in test_dataset[idx]['caption']] for idx in range(N_img)]

    d_emb_texscores = get_dict_clip_emb_texscores(clip_model, img_preproc, text, np.arange(m,M))
    end = time.time()
    print('time:', end - start)


    for i, (idx, df_boxes, caption) in enumerate(zip(d_emb_texscores.keys(), results, captions)):
        d_emb_texscores[idx]['df_boxes'] = df_boxes
        d_emb_texscores[idx]['caption'] = caption
        d_emb_texscores[idx]['bbox_target'] = test_dataset[i]['bbox']

    dictionary_full.update(d_emb_texscores)

    # # save checkpoint
    # with open(f'{base_path_save}yolo_v8x_1_dictionary_full_val_checkpoints.p', 'wb') as fp:
    #     pickle.dump(dictionary_full, fp, protocol=pickle.HIGHEST_PROTOCOL)

    # save checkpoint
    with open(f'{base_path_save}yolo_v8x_1_dictionary_full_train_checkpoints_28000.p', 'wb') as fp:
        pickle.dump(dictionary_full, fp, protocol=pickle.HIGHEST_PROTOCOL)

    # # save checkpoint
    # with open(f'{base_path_save}yolo_v5l6_1_dictionary_full_val_checkpoints.p', 'wb') as fp:
    #     pickle.dump(dictionary_full, fp, protocol=pickle.HIGHEST_PROTOCOL)

    # save checkpoint
    # with open(f'{base_path_save}yolo_v8x_1_dictionary_full_test_checkpoints.p', 'wb') as fp:
    #     pickle.dump(dictionary_full, fp, protocol=pickle.HIGHEST_PROTOCOL)


# with open(f'{base_path_save}yolo_v8x_1_dictionary_full_test.p', 'wb') as fp:
#     pickle.dump(dictionary_full, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'{base_path_save}yolo_v8x_1_dictionary_full_train_28000.p', 'wb') as fp:
    pickle.dump(dictionary_full, fp, protocol=pickle.HIGHEST_PROTOCOL)

# with open(f'{base_path_save}yolo_v8x_1_dictionary_full_val.p', 'wb') as fp:
#     pickle.dump(dictionary_full, fp, protocol=pickle.HIGHEST_PROTOCOL)

# with open(f'{base_path_save}yolo_v5l6_1_dictionary_full_val.p', 'wb') as fp:
#     pickle.dump(dictionary_full, fp, protocol=pickle.HIGHEST_PROTOCOL)

step: 28000 28016



0: 640x640 3 boats, 1: 640x640 2 persons, 4 teddy bears, 2: 640x640 1 truck, 2 dogs, 3: 640x640 1 cat, 5 umbrellas, 1 chair, 4: 640x640 1 person, 1 cat, 5: 640x640 16 persons, 3 horses, 6: 640x640 4 persons, 1 chair, 2 couchs, 4 remotes, 7: 640x640 4 persons, 4 bicycles, 1 bench, 2 handbags, 5 books, 8: 640x640 2 persons, 1 fork, 1 apple, 1 dining table, 9: 640x640 3 persons, 2 buss, 2 clocks, 10: 640x640 3 persons, 11: 640x640 3 mouses, 7 keyboards, 12: 640x640 2 persons, 1 fire hydrant, 2 umbrellas, 13: 640x640 2 airplanes, 1 truck, 14: 640x640 3 persons, 2 ties, 3 chairs, 1 dining table, 1 tv, 15: 640x640 4 persons, 2 cups, 15 bananas, 1 couch, 1 potted plant, 1 dining table, 1 cell phone, 1 vase, 842.8ms
Speed: 1.8ms preprocess, 52.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.d

time: 3.274167537689209
step: 28016 28033


0: 640x640 5 cows, 1: 640x640 2 tvs, 1 keyboard, 2 clocks, 2: 640x640 1 person, 3 trains, 3: 640x640 3 cars, 1 umbrella, 2 chairs, 4 books, 4: 640x640 3 persons, 4 surfboards, 5: 640x640 2 sandwichs, 1 dining table, 6: 640x640 2 persons, 1 spoon, 1 bowl, 3 chairs, 1 dining table, 7: 640x640 1 person, 2 kites, 2 couchs, 2 books, 1 scissors, 8: 640x640 2 persons, 1 frisbee, 9: 640x640 2 dogs, 4 chairs, 1 couch, 2 dining tables, 2 books, 10: 640x640 5 persons, 4 boats, 11: 640x640 4 persons, 1 backpack, 1 tie, 2 couchs, 12: 640x640 4 horses, 13: 640x640 3 persons, 8 sports balls, 4 tennis rackets, 14: 640x640 1 person, 1 car, 2 airplanes, 2 trucks, 15: 640x640 3 hot dogs, 1 dining table, 16: 640x640 6 persons, 1 dog, 2 horses, 1023.3ms
Speed: 1.9ms preprocess, 60.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is 

time: 2.5148837566375732
step: 28033 28050


...

step: 30868 30885


0: 640x640 3 persons, 1 bench, 1 tennis racket, 1: 640x640 2 persons, 1 handbag, 2 cups, 1 fork, 1 knife, 1 dining table, 2: 640x640 4 persons, 3 skateboards, 3: 640x640 5 boats, 2 dogs, 4: 640x640 4 persons, 4 knifes, 1 cake, 1 dining table, 5: 640x640 1 bottle, 3 chairs, 2 dining tables, 3 ovens, 1 sink, 1 refrigerator, 6: 640x640 1 bottle, 1 spoon, 3 bowls, 1 sandwich, 2 pizzas, 1 dining table, 7: 640x640 13 persons, 2 motorcycles, 1 umbrella, 1 handbag, 8: 640x640 4 persons, 2 airplanes, 1 truck, 9: 640x640 4 persons, 6 pizzas, 1 dining table, 10: 640x640 1 person, 4 cups, 4 sinks, 1 vase, 11: 640x640 2 traffic lights, 12: 640x640 3 giraffes, 13: 640x640 1 person, 2 spoons, 3 bowls, 2 refrigerators, 14: 640x640 11 persons, 3 cars, 6 motorcycles, 1 cow, 1 handbag, 15: 640x640 2 persons, 1 tie, 16: 640x640 1 person, 4 wine glasss, 1 cup, 1 chair, 1104.7ms
Speed: 1.4ms preprocess, 65.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'Boxes.boxes' is depre

time: 2.7152295112609863


step: 30885 30902


0: 640x640 2 persons, 1 sports ball, 3 tennis rackets, 1: 640x640 5 persons, 1 bed, 1 teddy bear, 2: 640x640 1 clock, 1 vase, 3: 640x640 1 person, 1 handbag, 2 bowls, 6 donuts, 4 chairs, 1 dining table, 1 laptop, 1 cell phone, 4: 640x640 3 persons, 4 cars, 1 bench, 1 handbag, 2 cups, 2 forks, 1 knife, 3 pizzas, 1 dining table, 1 remote, 5: 640x640 3 persons, 3 skiss, 6: 640x640 4 persons, 1 tie, 1 cell phone, 2 sinks, 7: 640x640 4 persons, 1 bench, 2 backpacks, 1 sports ball, 5 tennis rackets, 2 chairs, 8: 640x640 3 persons, 3 snowboards, 9: 640x640 2 persons, 3 bicycles, 3 dogs, 1 backpack, 10: 640x640 3 persons, 3 baseball gloves, 11: 640x640 2 persons, 2 boats, 2 ties, 2 chairs, 2 potted plants, 1 dining table, 12: 640x640 15 persons, 2 airplanes, 2 handbags, 13: 640x640 1 person, 1 skis, 14: 640x640 3 persons, 1 elephant, 15: 640x640 3 persons, 1 tie, 1 bottle, 16: 640x640 3 persons, 1 car, 2 trucks, 2 stop signs, 1098.4ms
